<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/langgraph_memory_semantic_email_assistant_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental" "aiosqlite"

In [ ]:
!pip install langmem

In [3]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
os.environ["TAVILY_API_KEY"] = userdata.get('tavily_api_key')

# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    #model="gemini-1.5-flash-001",
    temperature=0,
    verbose=True
)

StepI. Define classifications and store/memory

In [5]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team"
}
prompt_instructions = {
  "triage_rules": {
      "ignore": "Marketing newsletters, spam emails",
      "notify": "Team member out sick, build system notifications.",
      "respond": "Direct questions from team members, meeting requests."
  },
  "agent_instructions": "Use these tools when appropriate to help"
}

In [6]:
# Define output structure
from pydantic import BaseModel, Field
from typing import Literal
from langchain.chat_models import init_chat_model

class Router(BaseModel):
  """Analyze the unread email and route it according to its content"""

  reasoning: str = Field(
      description="Step-by-step reasoning behind the classification"
  )
  classification: Literal["ignore", "respond", "notify"] = Field(
      description="The classification of an email: 'ignore' for irrelavant emails, \
      'notify' for important information that doesn't need a response,\
      'respond' for emails that need a reply"
  )
llm_router = llm.with_structured_output(Router)

In [7]:
# Define in memory store and memory tools for semantic information
from langgraph.store.memory import InMemoryStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings

store = InMemoryStore(
    index={"embed": GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")}
)

from langmem import create_manage_memory_tool, create_search_memory_tool
manage_memory_tool = create_manage_memory_tool(
    namespace=(
        "email_assistant",
        "{langgraph_user_id}",
        "collection"
    )
)
search_memory_tool = create_search_memory_tool(
    namespace=(
        "email_assistant",
        "{langgraph_user_id}",
        "collection"
    )
)

In [ ]:
#print(manage_memory_tool)

name='manage_memory' description='Create, update, or delete a memory to persist across conversations.\nInclude the MEMORY ID when updating or deleting a MEMORY. Omit when creating a new MEMORY - it will be created for you.\nProactively call this tool when you:\n\n1. Identify a new USER preference.\n2. Receive an explicit USER request to remember something or otherwise alter your behavior.\n3. Are working and want to record important context.\n4. Identify that an existing MEMORY is incorrect or outdated.' args_schema=<class 'langchain_core.utils.pydantic.manage_memory'> func=<function create_manage_memory_tool.<locals>.manage_memory at 0x7adc00a56ca0> coroutine=<function create_manage_memory_tool.<locals>.amanage_memory at 0x7adc00a56b60>


In [8]:
# Define prompts schemas
triage_system_prompt = """
<Role>
You are {full_name}'s executive assistant designed to help manage incoming emails. You are \
a top-notch executive assistant who cares about {name} performing as well as possible.
</Role>

<Background>
{user_profile_background}
</Background>

<Instructions>

{name} gets lots of emails. Your job is to categorize each email into on of three\
categories:
1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but\
doesn't require a response
3. RESPOND - Emails that require a direct response from {name}

Classify the below email into one of these categories.

</Instructions>

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email\
response. For these, you should notify {name} (using the 'notofy' response). Examples \
of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >
{examples}
</ Few shot examples >
"""

triage_user_prompt = """
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}

"""

agent_system_prompt = """
<Role>
You are {full_name}'s executive assistant designed to help manage incoming emails. You are \
a top-notch executive assistant who cares about {name} performing as well as possible.
</Role>

<Tools>
You have access to the following tools to help mamage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_date) - Schedule calendar meetings
3. check_calender_availablity(day) - Check calender available time slots for a given day.
4. manage_memory - Store any relevant information about contacts, actions and so on
5. search_memory - Search for any relevant information
</Tools>

< Instructions >
{instructions}
</ Instructions >

"""

In [9]:
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "Jone Doe<John.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
    Hi John,

    I was reviewing the API documentation for the new authentication service but got some questions for several paths.

    Specifically, I'm looking at:
    - /auth/refresh
    - /auth/validate

    Thanks,
    Alice"""
}

Step II. Create response agent with tools to handle emails with response required

In [10]:
# Define tools (mocked behaviors)
from langchain_core.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
  """Write and send an email"""
  # Placeholder response - real app
  return f"Email send to: {to}\nSubject: {subject}\nBody: {content}"
@tool
def schedule_meetings(
    attendees: list[str],
    subject: str,
    duration_minutes: int,
    preferred_date: str
) -> str:
  """Schedule a calendar meeting with a list of attendees"""
  # Placeholder response - real app
  return f"Meeting '{subject}' scheduled for {preferred_date}"
@tool
def check_calender_availablity(day:str) -> str:
  """Check calender availability for a given day. """
  # Placeholder response - real app...
  return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"




In [11]:
def create_prompt(state):
  return [
      {
          "role": "system",
          "content": agent_system_prompt.format(
              instructions = prompt_instructions['agent_instructions'],
              **profile
          )
      }
  ] + state['messages']

In [12]:
# Create the agent
from langgraph.prebuilt import create_react_agent
tools = [write_email, schedule_meetings, check_calender_availablity, manage_memory_tool, search_memory_tool]
response_agent = create_react_agent(
    llm,
    tools,
    prompt = create_prompt,
    store = store
)

/tmp/ipython-input-3028261799.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  response_agent = create_react_agent(


In [ ]:
config = {"configurable": {"langgraph_user_id": "lance"}}
response = response_agent.invoke(
    {"messages": [{"role": "user", "content": "Jim is my friend"}]},
    config=config
)

In [ ]:
for m in response["messages"]:
  m.pretty_print()

================================ Human Message =================================

Jim is my friend
================================== Ai Message ==================================
Tool Calls:
  manage_memory (001fc471-15c9-4009-8c2e-8a2036260d7f)
 Call ID: 001fc471-15c9-4009-8c2e-8a2036260d7f
  Args:
    action: create
    content: Jim is John Doe's friend.
================================= Tool Message =================================
Name: manage_memory

created memory 935d79c0-6e2d-41a6-8afa-14ee028b1011
================================== Ai Message ==================================

Got it. I'll remember that Jim is your friend.


In [ ]:
response = response_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "who is Jim?"
    }],
}, config = config)
for m in response["messages"]:
  m.pretty_print()

================================ Human Message =================================

who is Jim?
================================== Ai Message ==================================
Tool Calls:
  search_memory (f881e761-450b-424a-b198-334f1e5352c3)
 Call ID: f881e761-450b-424a-b198-334f1e5352c3
  Args:
    query: Jim
================================= Tool Message =================================
Name: search_memory

[{"namespace":["email_assistant","lance","collection"],"key":"935d79c0-6e2d-41a6-8afa-14ee028b1011","value":{"content":"Jim is John Doe's friend."},"created_at":"2026-02-21T08:44:56.203390+00:00","updated_at":"2026-02-21T08:44:56.203395+00:00","score":0.703581329447393}]
================================== Ai Message ==================================

[{'type': 'text', 'text': "Jim is John Doe's friend.", 'extras': {'signature': 'Cr8BAb4+9vvR0BalKUISB+FKsJCV0RahbTdVeYKnSHdYFnq0fhFpJh9b1Hi0MBxxGr3znLRg7oBUapUlNKAOrRPUHR02rfq2I+65vWR8GiU/aCGUmqGEXkHtWk2xszSnGtPASlHMs2KRzO0TroDPphsv

In [ ]:
#store.search()

Step III. Create the whole email agent

In [ ]:
# Define state
from langgraph.graph import add_messages
from typing import TypedDict, Annotated

class State(TypedDict):
  email_input: dict
  messages: Annotated[list, add_messages]

In [ ]:
# Define the whole agent
from langgraph.graph import END, START, StateGraph
from langgraph.types import Command
from typing import Literal # Corrected import for Literal

def triage_router(state: State) -> dict: # Changed function name and return type
  author = state['email_input']['from']
  to = state['email_input']['to']
  subject = state['email_input']['subject']
  email_thread = state['email_input']['body']

  system_prompt = triage_system_prompt.format(
    full_name=profile['full_name'],
    name=profile['name'],
    user_profile_background=profile['user_profile_background'],
    examples = None,
    triage_no = prompt_instructions['triage_rules']['ignore'],
    triage_notify = prompt_instructions['triage_rules']['notify'],
    triage_email = prompt_instructions['triage_rules']['respond']
  )
  user_prompt = triage_user_prompt.format(
    author=author,
    to=to,
    subject=subject,
    email_thread=email_thread
  )
  result = llm_router.invoke(
      [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt},
      ]
  )

  classification_decision = result.classification
  print(f"Classification: {classification_decision}")

  if classification_decision == "respond":
    update = {
        "messages" : [
        {
            "role": "user",
            "content": f"Respond to the email {state['email_input']}"
        }]
    }
    goto = "response_agent"
  elif classification_decision == "ignore" or classification_decision == "notify":
    # No messages update for ignore/notify in this flow, just the classification result
    goto = END
    update = None
  else:
    raise ValueError(f"Invalid classification: {classification_decision}")

  return Command(goto=goto, update=update)

email_agent = StateGraph(State)
email_agent.add_node("triage_router_node", triage_router) # Node where classification happens
email_agent.add_node("response_agent", response_agent) # The agent node for responding

# Define the entry point
email_agent.add_edge(START, "triage_router_node")

# Add conditional edges from the triage_router_node
'''
email_agent.add_conditional_edges(
    "triage_router_node",  # The node whose output determines the next step
    route_classification,  # The function that routes based on state
    {
        "response_agent": "response_agent", # If route_classification returns "response_agent", go here
        END: END               # If route_classification returns END, end the graph
    }
)
'''
email_agent = email_agent.compile(store=store)

In [ ]:
response = email_agent.invoke({
    "email_input": email
}, config=config)

In [ ]:
for m in response["messages"]:
  m.pretty_print()

In [ ]:
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "Jone Doe<John.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
    Hi John,

    Any update on my previous ask?"""
}
response = email_agent.invoke({"email_input": email}, config=config)

In [ ]:
for m in response["messages"]:
  m.pretty_print()